## Sentiment Analysis

## Images

### Resnet

### VGG

In [1]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch

image_path = "/Users/I539205/Documents/dev/uni/sem2/TeamProject/Team_Project_ComputerVision/dataset/id_1079890077575204864_2019-01-01.jpg"
img = Image.open(image_path).convert("RGB")

model = timm.create_model('vgg19.tv_in1k', pretrained=True)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)


/Users/I539205/Documents/dev/uni/sem2/TeamProject/Team_Project_ComputerVision/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- deepmoji
- torchmoji

SentiNet

model does not work - search different one?

In [ ]:
from transformers import AutoModelForImageClassification, AutoTokenizer
from PIL import Image
import requests

# Load the model
model = AutoModelForImageClassification.from_pretrained("Ritvik19/sentinet-v1")
tokenizer = AutoTokenizer.from_pretrained("Ritvik19/sentinet-v1")

# Example image URL
image_path = "/Users/I539205/Documents/dev/uni/sem2/TeamProject/Team_Project_ComputerVision/dataset/id_1079890077575204864_2019-01-01.jpg"

# Preprocess the image
image = Image.open(requests.get(image_path, stream=True).raw)
inputs = tokenizer(image, return_tensors="pt")

# Classify the image
outputs = model(**inputs)
predicted_class = tokenizer.decode(outputs.logits.argmax())
print("Predicted sentiment:", predicted_class)


## VLM

introduction to VLMs: https://huggingface.co/blog/vision_language_pretraining#supporting-vision-language-models-in-%F0%9F%A4%97-transformers

### LENS + T5 
https://github.com/ContextualAI/lens

In [2]:
import requests
from lens import Lens, LensProcessor
from PIL import Image
import torch

image_path = "/Users/I539205/Documents/dev/uni/sem2/TeamProject/Team_Project_ComputerVision/dataset/id_1079890077575204864_2019-01-01.jpg"
image2_path = "/Users/I539205/Documents/dev/uni/sem2/TeamProject/Team_Project_ComputerVision/dataset/id_1079890804267610113_2019-01-01.jpg"
image = Image.open(image_path).convert("RGB")
question = "What is the sentiment of the image? Please classify the image' sentiment into positive, negative and neutral."

lens = Lens()
processor = LensProcessor()
with torch.no_grad():
    samples = processor([image],[question])
    output = lens(samples)
    prompts = output["prompts"]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small",truncation_side = 'left',padding = True)
LLM_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
# passes image description to llm and asks prompt
input_ids = tokenizer(samples["prompts"], return_tensors="pt").input_ids
outputs = LLM_model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> negative</s>


/Users/I539205/Documents/dev/uni/sem2/TeamProject/Team_Project_ComputerVision/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### ViLT -> visual QA

In [3]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import torch
import requests
from PIL import Image

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# download an input image
image = Image.open(image_path).convert("RGB")
text = "What is the sentiment of the image? Is the image' sentiment positive, negative or neutral?"

# prepare inputs
inputs = processor(image, text, return_tensors="pt")

# forward pass
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
idx = logits.argmax(-1).item()
print("Predicted answer:", model.config.id2label[idx])

Predicted answer: clear


### ViT

### BLIP


In [4]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

image = Image.open(image_path).convert("RGB")

question = "What is the image's sentiment? Is the image' sentiment positive, negative or neutral?"
inputs = processor(image, question, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))



positive


In [5]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-capfilt-large")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-capfilt-large")

image = Image.open(image_path).convert("RGB")

question = "What is the image's sentiment? Is the image' sentiment positive, negative or neutral?"
inputs = processor(image, question, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))



positive


### FLAVA

In [6]:
from PIL import Image
import requests

from transformers import FlavaProcessor, FlavaModel

model = FlavaModel.from_pretrained("facebook/flava-full")
processor = FlavaProcessor.from_pretrained("facebook/flava-full")

image = Image.open(image_path).convert("RGB")

inputs = processor(
  text=["What is the image's sentiment?"], images=[image], return_tensors="pt", padding="max_length", max_length=77
)

outputs = model(**inputs)
image_embeddings = outputs.image_embeddings # Batch size X (Number of image patches + 1) x Hidden size => 2 X 197 X 768
text_embeddings = outputs.text_embeddings # Batch size X (Text sequence length + 1) X Hidden size => 2 X 77 X 768
multimodal_embeddings = outputs.multimodal_embeddings # Batch size X (Number of image patches + Text Sequence Length + 3) X Hidden size => 2 X 275 x 768
# Multimodal embeddings can be used for multimodal tasks such as VQA

# TODO how can I use multimodal embeddings for sentiment analysis?


/Users/I539205/Documents/dev/uni/sem2/TeamProject/Team_Project_ComputerVision/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:1051: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


### Llava


In [7]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-34b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-34b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True) 
model.to("cuda:0")

# prepare image and text prompt, using the appropriate prompt template
question = "What is the sentiment of the image? Is the image' sentiment positive, negative or neutral?"
prompt = f"<|im_start|>system\nAnswer the questions.<|im_end|><|im_start|>user\n<image>\n{question}<|im_end|><|im_start|>assistant\n"

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=100)

print(processor.decode(output[0], skip_special_tokens=True))


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


KeyboardInterrupt: 

### CLIP

TODO: look into https://huggingface.co/flax-community/clip-vision-bert-vqa-ft-6k (needs download of model)

In [ ]:
from torchvision.io import read_image
import numpy as  np
import os
from transformers import CLIPProcessor, BertTokenizerFast
from model.flax_clip_vision_bert.modeling_clip_vision_bert import FlaxCLIPVisionBertForSequenceClassification
image_path = os.path.join('images/val2014', os.listdir('images/val2014')[0])
img = read_image(image_path)

clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')
clip_outputs = clip_processor(images=img)
clip_outputs['pixel_values'][0] = clip_outputs['pixel_values'][0].transpose(1,2,0) # Need to transpose images as model expected channel last images.

tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')
model = FlaxCLIPVisionBertForSequenceClassification.from_pretrained('flax-community/clip-vision-bert-vqa-ft-6k')

text = "Are there teddy bears in the image?"

tokens = tokenizer([text], return_tensors="np")
pixel_values = np.concatenate([clip_outputs['pixel_values']])
outputs = model(pixel_values=pixel_values, **tokens)
preds = outputs.logits[0]
sorted_indices = np.argsort(preds)[::-1] # Get reverse sorted scores
top_5_indices = sorted_indices[:5]
top_5_tokens = list(map(model.config.id2label.get,top_5_indices))
top_5_scores = preds[top_5_indices]
print(dict(zip(top_5_tokens, top_5_scores)))


### BERT

In [ ]:
from IPython.display import Image, display
import PIL.Image
import io
import torch
import numpy as np
from processing_image import Preprocess
from visualizing_image import SingleImageViz
from modeling_frcnn import GeneralizedRCNN
from utils import Config
import utils
from transformers import VisualBertForQuestionAnswering, BertTokenizerFast



URL = "https://vqa.cloudcv.org/media/test2014/COCO_test2014_000000262567.jpg"
OBJ_URL = "https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/genome/1600-400-20/objects_vocab.txt"
ATTR_URL = "https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/genome/1600-400-20/attributes_vocab.txt"
VQA_URL = "https://dl.fbaipublicfiles.com/pythia/data/answers_vqa.txt"

objids = utils.get_data(OBJ_URL)
attrids = utils.get_data(ATTR_URL)
vqa_answers = utils.get_data(VQA_URL)
# load models and model components
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")

frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg)

# image_preprocess = Preprocess(frcnn_cfg)

bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
visualbert_vqa = VisualBertForQuestionAnswering.from_pretrained("uclanlp/visualbert-vqa")


frcnn_visualizer = SingleImageViz(URL, id2obj=objids, id2attr=attrids)
# run frcnn
images, sizes, scales_yx = image_preprocess(URL)
output_dict = frcnn(
    images,
    sizes,
    scales_yx=scales_yx,
    padding="max_detections",
    max_detections=frcnn_cfg.max_detections,
    return_tensors="pt",
)
features = output_dict.get("roi_features")


test_question = "What is the image's sentiment? Is the image' sentiment positive, negative or neutral?"

inputs = bert_tokenizer(
    test_question,
    padding="max_length",
    max_length=20,
    truncation=True,
    return_token_type_ids=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt",
)

output_vqa = visualbert_vqa(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    visual_embeds=features,
    visual_attention_mask=torch.ones(features.shape[:-1]),
    token_type_ids=inputs.token_type_ids,
    output_attentions=False,
)
# get prediction
pred_vqa = output_vqa["logits"].argmax(-1)
print("Question:", test_question)
print("prediction from VisualBert VQA:", vqa_answers[pred_vqa])

### GIT

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from PIL import Image

processor = AutoProcessor.from_pretrained("microsoft/git-base-textvqa")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-textvqa")

image = Image.open(image_path).convert("RGB")

pixel_values = processor(images=image, return_tensors="pt").pixel_values

question = "Which of these categories 'positive', 'negative' or 'neutral' best describes the image sentiment and feeling?"

input_ids = processor(text=question, add_special_tokens=False).input_ids
input_ids = [processor.tokenizer.cls_token_id] + input_ids
input_ids = torch.tensor(input_ids).unsqueeze(0)

generated_ids = model.generate(pixel_values=pixel_values, input_ids=input_ids, max_length=50)
print(processor.batch_decode(generated_ids, skip_special_tokens=True))

Other options:  VisionEncoderDecoderModel, VisionTextDualEncoder